<a href="https://colab.research.google.com/github/joeyudongs/google-docs-markdown-formatter/blob/master/Markdown_To_GoogleDoc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!pip install --upgrade google-api-python-client google-auth google-auth-httplib2 google-auth-oauthlib


In [34]:
import re
import json
import google.auth
from googleapiclient.discovery import build
from google.colab import auth

def authenticate_user():
    auth.authenticate_user()
    creds, _ = google.auth.default()
    return creds

def get_bullet_level(line):
    """
    Calculate nesting level based on number of leading spaces.
    EXAMPLE:
      - 0 spaces => level 1
      - 2 spaces => level 2
      - 4 spaces => level 3
      etc.

    The +1 ensures that top-level bullets (no leading spaces)
    get at least 'level 1' indentation in Google Docs.
    """
    leading_spaces = len(line) - len(line.lstrip())
    return (leading_spaces // 2) + 1

def format_content(markdown_text):
    lines = markdown_text.split('\n')
    requests = []
    index = 1  # Insert text starting at position 1

    # Regex to find all @mentions in a line (e.g. @sarah, @mike)
    mention_pattern = re.compile(r'@\w+')

    for line in lines:
        raw_line = line.rstrip('\r')
        stripped_line = raw_line.strip()

        # Heading 1
        if stripped_line.startswith("# "):
            text = stripped_line[2:].strip()
            requests.append({
                "insertText": {
                    "location": {"index": index},
                    "text": text + "\n"
                }
            })
            requests.append({
                "updateParagraphStyle": {
                    "range": {
                        "startIndex": index,
                        "endIndex": index + len(text) + 1
                    },
                    "paragraphStyle": {"namedStyleType": "HEADING_1"},
                    "fields": "namedStyleType"
                }
            })
            index += len(text) + 1

        # Heading 2
        elif stripped_line.startswith("## "):
            text = stripped_line[3:].strip()
            requests.append({
                "insertText": {
                    "location": {"index": index},
                    "text": text + "\n"
                }
            })
            requests.append({
                "updateParagraphStyle": {
                    "range": {
                        "startIndex": index,
                        "endIndex": index + len(text) + 1
                    },
                    "paragraphStyle": {"namedStyleType": "HEADING_2"},
                    "fields": "namedStyleType"
                }
            })
            index += len(text) + 1

        # Heading 3
        elif stripped_line.startswith("### "):
            text = stripped_line[4:].strip()
            requests.append({
                "insertText": {
                    "location": {"index": index},
                    "text": text + "\n"
                }
            })
            requests.append({
                "updateParagraphStyle": {
                    "range": {
                        "startIndex": index,
                        "endIndex": index + len(text) + 1
                    },
                    "paragraphStyle": {"namedStyleType": "HEADING_3"},
                    "fields": "namedStyleType"
                }
            })
            index += len(text) + 1

        # Checkbox bullet (e.g. - [ ])
        elif stripped_line.startswith("- [ ]"):
            content = stripped_line[5:].strip()
            level = get_bullet_level(raw_line)

            requests.append({
                "insertText": {
                    "location": {"index": index},
                    "text": content + "\n"
                }
            })
            # Create bullet as checkbox
            requests.append({
                "createParagraphBullets": {
                    "range": {
                        "startIndex": index,
                        "endIndex": index + len(content) + 1
                    },
                    "bulletPreset": "BULLET_CHECKBOX"
                }
            })
            # Indent based on nesting level
            requests.append({
                "updateParagraphStyle": {
                    "range": {
                        "startIndex": index,
                        "endIndex": index + len(content) + 1
                    },
                    "paragraphStyle": {
                        # 36 points = 0.5 inch. Adjust if you want more/less indentation.
                        "indentStart": {
                            "magnitude": 36 * level,
                            "unit": "PT"
                        }
                    },
                    "fields": "indentStart"
                }
            })
            # Check if there are any @mentions inside
            for mention_match in mention_pattern.finditer(content):
                mention_start = index + mention_match.start()
                mention_end = index + mention_match.end()
                requests.append({
                    "updateTextStyle": {
                        "range": {
                            "startIndex": mention_start,
                            "endIndex": mention_end
                        },
                        "textStyle": {
                            "bold": True,
                            "foregroundColor": {
                                "color": {
                                    "rgbColor": {"red": 0.0, "green": 0.0, "blue": 1.0}
                                }
                            }
                        },
                        "fields": "bold,foregroundColor"
                    }
                })
            index += len(content) + 1

        # Regular bulleted item (leading '*')
        elif stripped_line.startswith("*"):
            level = get_bullet_level(raw_line)
            # Remove leading '*' and extra space
            content = stripped_line[1:].strip()

            requests.append({
                "insertText": {
                    "location": {"index": index},
                    "text": content + "\n"
                }
            })
            # Create bullet
            requests.append({
                "createParagraphBullets": {
                    "range": {
                        "startIndex": index,
                        "endIndex": index + len(content) + 1
                    },
                    "bulletPreset": "BULLET_DISC_CIRCLE_SQUARE"
                }
            })
            # Indent bullet based on nesting level
            requests.append({
                "updateParagraphStyle": {
                    "range": {
                        "startIndex": index,
                        "endIndex": index + len(content) + 1
                    },
                    "paragraphStyle": {
                        "indentStart": {
                            "magnitude": 36 * level,
                            "unit": "PT"
                        }
                    },
                    "fields": "indentStart"
                }
            })
            # Check for @mentions
            for mention_match in mention_pattern.finditer(content):
                mention_start = index + mention_match.start()
                mention_end = index + mention_match.end()
                requests.append({
                    "updateTextStyle": {
                        "range": {
                            "startIndex": mention_start,
                            "endIndex": mention_end
                        },
                        "textStyle": {
                            "bold": True,
                            "foregroundColor": {
                                "color": {
                                    "rgbColor": {"red": 0.0, "green": 0.0, "blue": 1.0}
                                }
                            }
                        },
                        "fields": "bold,foregroundColor"
                    }
                })
            index += len(content) + 1

        # Horizontal rule / special footer marker (---)
        elif stripped_line.startswith("---"):
            requests.append({
                "insertText": {
                    "location": {"index": index},
                    "text": stripped_line + "\n"
                }
            })
            requests.append({
                "updateTextStyle": {
                    "range": {
                        "startIndex": index,
                        "endIndex": index + len(stripped_line)
                    },
                    "textStyle": {
                        "italic": True,
                        "foregroundColor": {
                            "color": {
                                "rgbColor": {"blue": 0.5}
                            }
                        }
                    },
                    "fields": "italic,foregroundColor"
                }
            })
            index += len(stripped_line) + 1

        # Footer lines (distinct style if they start with "Meeting recorded by:" or "Duration:")
        elif stripped_line.startswith("Meeting recorded by:") or stripped_line.startswith("Duration:"):
            # Insert the line
            requests.append({
                "insertText": {
                    "location": {"index": index},
                    "text": stripped_line + "\n"
                }
            })
            # Apply distinct style (e.g., italic + gray)
            requests.append({
                "updateTextStyle": {
                    "range": {
                        "startIndex": index,
                        "endIndex": index + len(stripped_line)
                    },
                    "textStyle": {
                        "italic": True,
                        "foregroundColor": {
                            "color": {
                                "rgbColor": {"red": 0.4, "green": 0.4, "blue": 0.4}
                            }
                        }
                    },
                    "fields": "italic,foregroundColor"
                }
            })
            index += len(stripped_line) + 1

        # Plain text (including lines with possible @mentions)
        else:
            if stripped_line:
                text_to_insert = stripped_line
            else:
                text_to_insert = ""

            requests.append({
                "insertText": {
                    "location": {"index": index},
                    "text": text_to_insert + "\n"
                }
            })
            # Apply mention styling for each @ mention in the line
            for mention_match in mention_pattern.finditer(text_to_insert):
                mention_start = index + mention_match.start()
                mention_end = index + mention_match.end()
                requests.append({
                    "updateTextStyle": {
                        "range": {
                            "startIndex": mention_start,
                            "endIndex": mention_end
                        },
                        "textStyle": {
                            "bold": True,
                            "foregroundColor": {
                                "color": {
                                    "rgbColor": {"red": 0.0, "green": 0.0, "blue": 1.0}
                                }
                            }
                        },
                        "fields": "bold,foregroundColor"
                    }
                })
            index += len(text_to_insert) + 1

    return requests

def create_google_doc(creds, title, content):
    try:
        service = build('docs', 'v1', credentials=creds)
        # Create a new Google Doc
        doc = service.documents().create(body={"title": title}).execute()
        document_id = doc.get('documentId')

        # Generate the requests and apply to the new doc
        requests = format_content(content)
        service.documents().batchUpdate(documentId=document_id, body={"requests": requests}).execute()

        print(f"Document created successfully:")
        print(f"https://docs.google.com/document/d/{document_id}")
    except Exception as e:
        print(f"Error creating Google Doc: {e}")

def main():
    markdown_notes = """# Product Team Sync - May 15, 2023

## Attendees
- Sarah Chen (Product Lead)
- Mike Johnson (Engineering)
- Anna Smith (Design)
- David Park (QA)

## Agenda

### 1. Sprint Review
* Completed Features
  * User authentication flow
  * Dashboard redesign
  * Performance optimization
    * Reduced load time by 40%
    * Implemented caching solution
* Pending Items
  * Mobile responsive fixes
  * Beta testing feedback integration

### 2. Current Challenges
* Resource constraints in QA team
* Third-party API integration delays
* User feedback on new UI
  * Navigation confusion
  * Color contrast issues

### 3. Next Sprint Planning
* Priority Features
  * Payment gateway integration
  * User profile enhancement
  * Analytics dashboard
* Technical Debt
  * Code refactoring
  * Documentation updates

## Action Items
- [ ] @sarah: Finalize Q3 roadmap by Friday
- [ ] @mike: Schedule technical review for payment integration
- [ ] @anna: Share updated design system documentation
- [ ] @david: Prepare QA resource allocation proposal

## Next Steps
* Schedule individual team reviews
* Update sprint board
* Share meeting summary with stakeholders

## Notes
* Next sync scheduled for May 22, 2023
* Platform demo for stakeholders on May 25
* Remember to update JIRA tickets

---
Meeting recorded by: Sarah Chen
Duration: 45 minutes
"""

    creds = authenticate_user()
    create_google_doc(creds, "Product Team Sync - May 15, 2023", markdown_notes)

if __name__ == "__main__":
    main()


Document created successfully:
https://docs.google.com/document/d/1Sq3UbqragG2GfQe3wEqxGFQC6Yj9QLWvXI_-jLLXVS4
